## Overview
Here we use ASPECT to numerically reproduce the results of a linear stability analysis for the onset of convection in a fluid layer heated from below. This exercise was assigned to students in a geodynamics class at Portland State University as a first step towards setting up a nominally Earth-like mantle convection model. Hence, representative length scales and transport properties for Earth are used.

The linear stability analysis appears in Turcotte and Schubert (2014) section 6.19. To use this code, you must compile aspect and give the path to the executable below as $\texttt{aspect_bin}$. The critical Rayleigh number for the onset of convection depends only on the dimensionless wavelength of the perturbation, which is assumed to be equal to width of the domain. The domain has height $b$ and width $\lambda$ and the perturbation has the functional form:

$
T'(x,y) = T_0'\cos\left(\frac{2\pi x}{\lambda}\right)\sin\left(\frac{\pi y}{b} \right)
$

Note that because we place the bottom boundary of the domain at $y=0$ and the top at $y=b$, the perturbation vanishes at the top and bottom boundaries.

The analytic solution for the critical Rayleigh number, $Ra_{cr}$ is given in T\&S equation 6.319:

$
Ra_{cr}=\frac{\left(\pi^2+\frac{4\pi^2 b^2}{\lambda^2}\right)^3}{\frac{4\pi^2 b^2}{\lambda^2}}
$

The linear stability analysis also makes a prediction for the dimensionless growth rate of the instability $\alpha'$. The maximum vertical velocity is given by:

$
v_{y,max} = \frac{2\pi}{\lambda}\phi_0' e^{\alpha' t}
$,

where

$
\phi_0' = -\frac{2\pi}{\lambda}\frac{\rho_0 g \alpha T_0'}{\mu}\left(\frac{4\pi^2}{\lambda^2}+\frac{\pi^2}{b^2} \right)^{-2}
$,

and

$
\alpha'=\frac{\kappa}{b^2}\left[\frac{\rho_0 g \alpha b^3 \Delta T}{\mu \kappa}\left(\frac{\frac{4\pi^2 b^2}{\lambda^2}}{\left(\frac{4\pi^2 b^2}{\lambda^2}+\pi^2\right)^2}\right) -\left(\pi^2+\frac{4\pi^2b^2}{\lambda^2}\right)\right]
$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from subprocess import run

base_input = "edge-isoviscous-base.prm"     # The 'base' input file that gets modified
input_file = "input.prm"                   # The temporary input file that is used for each run
aspect_bin = "../../build/aspect"          # Path to aspect executable
output_dir = "output"                      # Output directory

In [2]:
# Create a dictionary to replace keys in base input file with appropriate values
parameters = dict([])
parameters['RAYLEIGH'] = 0.0          # vertical gravitational acceleration

def generate_input_file(base_file_name,output_file_name,dictionary):
    """Read the 'base' input file from base_file_name, replace strings 
    using dictionary, and write new output file to output_file_name"""
    fh = open(base_file_name,'r')
    run(['rm','-f',output_file_name])
    ofh = open(output_file_name,'w')
    for line in fh:        
        for key in dictionary:
            if key in line:                
                line = line.replace(key,str(dictionary[key]))
        ofh.write(line)
    fh.close()
    ofh.close()
    
def parse_output(output_dir):
    """Read the statistics (stats_file) file from the output directory"""
    tmp = np.loadtxt(output_dir + '/statistics',usecols=(1, 11,21))
    #vy = np.loadtxt(output_dir + '/point_values.txt',usecols=(4,))
    t=tmp[:,0]
    vmax = tmp[:,1]
    return t,vmax,vy,qsurf
    
def vy_exact(t,p):
    """Evaluate exact expression for maximum upward velocity"""
    lam = p['WIDTH']
    b = p['HEIGHT']
    g = p['GRAVITY']
    T0 = p['PMAG']
    alpha = p['ALPHA']
    eta = p['VISCOSITY']
    rho = p['DENSITY']
    dT = p['DELTA_T']
    kappa = p['KTHERMAL']/rho/p['SPECIFIC_HEAT']
    alpha_prime = kappa/b**2*((rho*g*alpha*b**3*dT/eta/kappa)*(4.0*np.pi**2*b**2/lam**2/(4.0*np.pi**2*b**2/lam**2+np.pi**2)**2)-(np.pi**2+4.0*np.pi**2*b**2/lam**2))
    phi0 = -2.0*np.pi/lam*(rho*g*alpha*T0/eta)*(4.*np.pi**2/lam**2 + np.pi**2/b**2)**(-2.0)
    vymax = 2.0*np.pi/lam*phi0 * np.exp(alpha_prime*t)
    return vymax
    
def run_aspect(rayleigh,p=parameters):
    """Perofm the following tasks. 
    1. Choose viscosity to satisfy rayleigh number rayleigh and parameters in p, which
    defaults to the parameters dictionary defined in the current namespace. 
    2. Generate an input file. 
    3. Run aspect
    4. Parse the results, return the rate at which velocities increase between the first and second timesteps."""
    # def eta(ra):        
    #     rho = p['DENSITY']
    #     alpha = p['ALPHA']
    #     g = p['GRAVITY']
    #     dT = p['DELTA_T']
    #     h = p['HEIGHT']     
    #     kappa = p['KTHERMAL']/p['DENSITY']/p['SPECIFIC_HEAT']    
    #     return rho*g*alpha*dT*(h**3)/ra/kappa                
    
    p['RAYLEIGH'] = rayleigh;    
    generate_input_file(base_input,input_file,p)
    run(['rm','-rf',output_dir])
    run([aspect_bin,input_file])
    t, vmax, vy = parse_output(output_dir)
    return (vmax[1]-vmax[0]), t, vy



In [12]:
rayleigh_numbers = 10.0 ** np.linspace(0,5,11)
print(rayleigh_numbers)

[1.00000000e+00 3.16227766e+00 1.00000000e+01 3.16227766e+01
 1.00000000e+02 3.16227766e+02 1.00000000e+03 3.16227766e+03
 1.00000000e+04 3.16227766e+04 1.00000000e+05]


In [ ]:
for ra in 